In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

from time import sleep, time

from tqdm import tqdm

import os

## Opening Browser

In [7]:
width = 1920
height = 1080

BASE = "https://www.nasdaq.com"

In [8]:
options = webdriver.ChromeOptions()
# Redirect downloads to particular directory in case of link
options.add_experimental_option("prefs", {
  "profile.default_content_settings.popups": 0,
  "download.prompt_for_download": False,
  "download.default_directory": r"~/Desktop/StockData", # This doesn't work with buttons, set download location manually on browser
})

driver = webdriver.Chrome(options=options)
driver.set_window_size(width , height)

## Visiting website and accessing NASDAQ stock data

In [10]:
driver.get(BASE+"/market-activity/quotes/nasdaq-ndx-index")

In [11]:
stocks = driver.find_elements(By.XPATH,'/html/body/div[2]/div/main/div[2]/article/div[2]/div/div[3]/div[3]/div[2]/table/tbody/tr/th[1]/a')

stock_links = dict()
for stock in stocks:
    #print(stock.text)
    
    stock_links[stock.text] = stock.get_attribute('href')
    
stock_links

{'AAPL': 'https://www.nasdaq.com/market-activity/stocks/aapl',
 'ABNB': 'https://www.nasdaq.com/market-activity/stocks/abnb',
 'ADBE': 'https://www.nasdaq.com/market-activity/stocks/adbe',
 'ADI': 'https://www.nasdaq.com/market-activity/stocks/adi',
 'ADP': 'https://www.nasdaq.com/market-activity/stocks/adp',
 'ADSK': 'https://www.nasdaq.com/market-activity/stocks/adsk',
 'AEP': 'https://www.nasdaq.com/market-activity/stocks/aep',
 'AMAT': 'https://www.nasdaq.com/market-activity/stocks/amat',
 'AMD': 'https://www.nasdaq.com/market-activity/stocks/amd',
 'AMGN': 'https://www.nasdaq.com/market-activity/stocks/amgn',
 'AMZN': 'https://www.nasdaq.com/market-activity/stocks/amzn',
 'ANSS': 'https://www.nasdaq.com/market-activity/stocks/anss',
 'ASML': 'https://www.nasdaq.com/market-activity/stocks/asml',
 'AVGO': 'https://www.nasdaq.com/market-activity/stocks/avgo',
 'AZN': 'https://www.nasdaq.com/market-activity/stocks/azn',
 'BIIB': 'https://www.nasdaq.com/market-activity/stocks/biib',
 '

## Downloading each stock data

In [13]:
def download_dataset(link):
    # click button labelled "max" to get 10 years of data
    driver.get(link+"/historical")
    
    button = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH,'/html/body/div[2]/div/main/div[2]/div[4]/div[3]/div/div[1]/div/div[1]/div[3]/div/div/div/button[6]')))
    button.click()
    
    driver.find_element(By.XPATH,'/html/body/div[2]/div/main/div[2]/div[4]/div[3]/div/div[1]/div/div[1]/div[3]/button').click()
    sleep(3)

In [14]:
for link in tqdm(list(stock_links.values())):
    download_dataset(link)

100%|██████████| 101/101 [13:14<00:00,  7.87s/it]


In [32]:
driver.quit()

## Renaming Files

In [31]:
directory = 'DATA_10y/'
files = os.listdir(directory)
files.sort(key=lambda x: os.path.getmtime(os.path.join(directory, x)))

for file, stock in zip(files[1:], stock_links.keys()):
    if file != ".DS_Store":
        old_path = os.path.join(directory, file)
        new_path = os.path.join(directory, stock + ".csv")

        os.rename(old_path, new_path)
